In [ ]:
import random
import time
import os

random.seed(time.time())

# baseBet = 2
# sat = 10
# mult = 2
# prob = 48
# ink = 0.1

params = [[1.01, 94.06], [1.05, 90.48], [1.1, 86.36], [1.2, 79.17], [1.3, 73.08], [1.4, 67.86], [1.5, 63.33], 
[1.6, 59.89], [1.7, 55.88], [1.8, 52.78], [1.9, 50], [2, 47.5], [2.2, 43.18], [2.4, 38.58], [2.6, 36.54],
[3, 31.67], [3.5, 27.14], [4, 23.75], [4.5, 20.65], [5, 19], [6, 15.83], [7, 13.57], [8, 11.88], [9, 10.56], 
[10, 9.5], [11.99, 7.92], [13.99, 6.79], [15.01, 6.33], [20, 4.75], [25, 3.8], [30, 3.17], [40, 2.38],
[50, 1.9], [100, 0.95]]
# print(params)
def doTheThing(prob, ink, baseBet, mult, baseSat):
    countLose = 0
    maxLose = 0
    for a in range(200):
        # os.system('cls' if os.name == 'nt' else 'clear')
        # print("started iter #" + str(a))

        hi = True
        
        def isGood(flip):
            if hi:
                return flip > 10000 - prob * 100
            return flip < prob * 100
        sat = baseSat
        bet = baseBet
        maxMinus = 0
        maxStreak = 0
        currentMinus = 0
        maxBet = 0
        maxBalance = 0
        minBet = 1
        minBalance = sat
        minB = baseBet
        currentStreak = 0
        
        for i in range(0, 10000):
            random.seed(time.time())
            if sat > maxBalance:
                maxBalance = sat
            
            if minBalance > sat:
                minBalance = sat
                
            if minB > bet:
                minB = bet
            
            if sat < bet:
                countLose += 1
                # print("End at " + str(i) + " iter.")
                break
            sat -= bet
            
            if bet > maxBet:
                maxBet = bet
            
            currentMinus -= bet
            
            if abs(currentMinus) > abs(maxMinus):
                maxMinus = currentMinus
                
            
            flip = random.randint(0, 10000)
            if isGood(flip):
                sat += bet * mult
                bet = baseBet
                currentMinus = 0
                currentStreak = 0
            else:
                bet = round(bet * (ink))
                if bet < minBet:
                    bet = minBet
                currentStreak += 1
                if currentStreak > maxStreak:
                    maxStreak = currentStreak
            if sat > 100000:
                
                print("got it with")
                print("baseBet: " + str(baseBet))
                print("prob: " + str(prob))
                print("ink: " + str(ink))
                print("mult: " + str(mult))
                print("in: " + str(a))
                break
            
        # print("balance: " + str(sat))
        # print("maxMinus :" + str(maxMinus))
        # print("maxStreak: " + str(maxStreak))
        # print("maxBet: " + str(maxBet))
        # print("maxBalance: " + str(maxBalance))
        # print("minBet: " + str(minB))
        # print("minBalance: " + str(minBalance))
        # print()

        if sat < baseSat * 1.1:
            countLose += 1
        if abs(maxMinus) > abs(maxLose):
            maxLose = maxMinus
    
    # print("countLose: " + str(countLose))
    # print("maxLose: " + str(maxLose))
    if(countLose == 0):
        print("totally got it with")
        print("baseBet: " + str(baseBet))
        print("prob: " + str(prob))
        print("ink: " + str(ink))
        print("mult: " + str(mult))
        # print("last sat: " + str(sat))
        print("baseSat: " + str(baseSat))
        print("maxLose: " + str(maxLose))
        
    
for p in params:
    print("starting with param: ")
    print(p)
    for cink in range(1, 100000, 1):
        for cs in range(1, 200, 1):
            for sat in range(50, 1000, 50):
                doTheThing(p[0], cink/10, cs, p[1], sat)
        

In [ ]:
import multiprocessing

num_cores = multiprocessing.gpu_count()
print(num_cores)


In [ ]:
import numpy as np
import pandas as pd 
import math, re, os
import random
import gc
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
print("Tensorflow version " + tf.__version__)
from sklearn.metrics import accuracy_score
from collections import Counter
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
class FixedDropout(tf.keras.layers.Dropout):
    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape
        symbolic_shape = K.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape
        for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)

In [ ]:
dense201 = tf.keras.models.load_model('../input/train-model-cassava/densenet201.h5')
inception = tf.keras.models.load_model('../input/train-model-cassava/inceptionv3.h5')
efficient_net = tf.keras.models.load_model(
    '../input/train-model-cassava/efficient_netb3.h5',
    compile=False,
    custom_objects={'FixedDropout':FixedDropout}
)

In [ ]:
JPEG_PATH = "../input/cassava-leaf-disease-classification/test_images"

import cv2

def load_image(jpeg_path, image_id):
    img = cv2.imread(os.path.join(jpeg_path, image_id))/255.0
    img = cv2.resize(img, (512, 512))[:, :, ::-1]

    return img
def generator(filepath, paths, batch_size=32):
    i=0
    print(len(paths))
    while i <= len(paths):
        batch = []
        for cpt in range(batch_size):
            if i + cpt >= len(paths):
                i += batch_size
                break
            batch.append(load_image(filepath, paths[i+cpt]))
            
        i += batch_size
        yield np.stack(batch)

In [ ]:
submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

In [ ]:
def vote_in_ensemble(v1, v2, v3):
    if v1 == v2:
        return v1
    if v2 == v3:
        return v2
    if v1 == v3:
        return v3
    return v1

In [ ]:
def predict_for_pretrained(model):
    ds_test = generator(JPEG_PATH,np.sort(submission.image_id.values)) 
    preds = np.argmax(model.predict(ds_test, verbose=True), axis=-1)
    return preds

dense_preds = predict_for_pretrained(dense201)
inception_preds = predict_for_pretrained(inception)
efficient_net_preds = predict_for_pretrained(efficient_net)

result = []

for idx, val in enumerate(dense_preds):
    result.append(
        vote_in_ensemble(
            dense_preds[idx],
            inception_preds[idx],
            efficient_net_preds[idx]
        )
    )

In [ ]:
submission["label"] = result
submission.to_csv("submission.csv", index=False)